### 安装 <a href="http://tushare.org/index.html">tushare</a> 
    pip install tushare
 

In [13]:
import tushare as ts
df = ts.get_k_data('002408', ktype='5',autype="qfq",index=False)

df = ts.get_tick_data('002408',date='2018-05-14')
df.head(100)


,time,price,change,volume,amount,type
0,15:00:03,14.09,0.01,930,1310370,买盘
1,14:57:00,14.08,--,4,5632,卖盘
2,14:56:54,14.08,-0.01,158,222464,卖盘
3,14:56:51,14.09,--,65,91585,买盘
4,14:56:48,14.09,0.01,226,318434,买盘
5,14:56:45,14.08,--,78,109824,卖盘
6,14:56:39,14.08,--,37,52096,买盘
7,14:56:33,14.08,--,161,226688,卖盘
8,14:56:30,14.08,0.01,57,80256,买盘
9,14:56:27,14.07,-0.01,15,21105,买盘
